# DPGP3 diversity analysis

The goal of this notebook is to analyze the DPGP3 data, looking at the coarse-grained two-site frequency spectrum for signatures of non-Kingman coalescence.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import sys
sys.path.insert(0, "/home/dpr/mmc_genomics/src")
import helpers as h

ImportError: No module named 'helpers'

# Data

## Diversity

In [ ]:
data_dir = '/project/jnovembre/data/external_public/DPGP3/'
chromosomes = ['2L', '2R', '3L', '3R']
files = {c : data_dir + 'Chr' + c + '.mac.txt.gz' for c in chromosomes}
chrom_lengths = {'2L':23011544,
                 '2R':21146708,
                 '3L':24543557,
                 '3R':27905053}
#files = [data_dir + 'Chr' + c + '.mac.txt.gz' for c in chromosomes]
#chrom_lengths = [23011544, 21146708, 24543557, 27905053]

In [ ]:
data = {c : h.loadints(files[c], chrom_lengths[c], 2) for c in chromosomes}

In [ ]:
nobs = {c:data[c][:,0] for c in chromosomes}
mac = {c:data[c][:,1] for c in chromosomes}

## 4-fold degenerate sites

In [ ]:
fourD_sites = pd.read_table('../data/dmel-4Dsites.txt', header=None, names=['chr', 'pos'])
fourD_sites.head()

In [ ]:
#Convert from one-index to zero-index
fourD_pos = {chrom : np.array(fourD_sites.pos[fourD_sites.chr == chrom] - 1) for chrom in chromosomes}

In [ ]:
print('Fraction of sites that are 4-fold degenerate:')
for chrom in chromosomes:
    print('{}\t{:.3f}'.format(chrom, len(fourD_pos[chrom]) / data[chrom].shape[0]))

## Exomes

To-do.

# Coverage and diversity

In [ ]:
window_size = 1000

plt.figure(figsize=(15,10))
for i, chrom in enumerate(chromosomes):
    n_w = chrom_lengths[chrom] // window_size
    pos = np.arange(n_w)*window_size / 1e6
    nobs_w = np.mean(nobs[chrom][:n_w*window_size].reshape(n_w, window_size), axis=1)
                                 
    plt.subplot(2,2,i+1)
    plt.plot(pos, nobs_w, '.', ms=1, alpha=0.1)#, linestyle='steps-pre')

    plt.title(chrom)
    if i%2 == 0:
        plt.ylabel('Avg. number of called genotypes')
    if i >= 2:
        plt.xlabel('Position (Mb)')
plt.show()

In [ ]:
window_size = 10000

plt.figure(figsize=(15,10))
for i, chrom in enumerate(chromosomes):
    n_w = chrom_lengths[chrom] // window_size
    pos = np.arange(n_w)*window_size / 1e6
    nobs_w = np.mean(nobs[chrom][:n_w*window_size].reshape(n_w, window_size), axis=1)
                                 
    plt.subplot(2,2,i+1)
    plt.plot(pos, nobs_w, '.', ms=1, alpha=1)#, linestyle='steps-pre')
    plt.ylim([85,101])
    plt.title(chrom)
    if i%2 == 0:
        plt.ylabel('Avg. number of called genotypes')
    if i >= 2:
        plt.xlabel('Position (Mb)')
plt.show()

In [ ]:
window_size = 10000

plt.figure(figsize=(15,10))
for i, chrom in enumerate(chromosomes):
    n_w = chrom_lengths[chrom] // window_size
    pos = np.arange(n_w)*window_size / 1e6

    hascov = nobs[chrom] > 0
    hascov_w = np.sum(hascov[:n_w*window_size].reshape(n_w, window_size), axis=1)

    poly = mac[chrom] > 0
    poly_w = np.sum(poly[:n_w*window_size].reshape(n_w, window_size), axis=1)
                             
    plt.subplot(2,2,i+1)
    plt.plot(pos, poly_w/hascov_w, '.', ms=1, alpha=1)#, linestyle='steps-pre')
    plt.ylim([-0.005,0.145])
    plt.title(chrom)
    if i%2 == 0:
        plt.ylabel('Fraction of polymorphic sites')
    if i >= 2:
        plt.xlabel('Position (Mb)')
plt.show()

In [ ]:
window_size = 10000

plt.figure(figsize=(15,10))
for i, chrom in enumerate(chromosomes):
    n_w = chrom_lengths[chrom] // window_size
    pos = np.arange(n_w)*window_size / 1e6
    
    hascov = nobs[chrom] > 0
    hascov_w = np.sum(hascov[:n_w*window_size].reshape(n_w, window_size), axis=1)
    
    pi = h.pairwise_diversity(mac[chrom], nobs[chrom])
    pi_w = np.nansum(pi[:n_w*window_size].reshape(n_w, window_size), axis=1)
                             
    plt.subplot(2,2,i+1)
    plt.plot(pos, pi_w/hascov_w, '.', ms=1, alpha=1)#, linestyle='steps-pre')

    plt.ylim([-0.001, 0.021])
    plt.title(chrom)
    if i%2 == 0:
        plt.ylabel('Average pairwise diversity')
    if i >= 2:
        plt.xlabel('Position (Mb)')
plt.show()

In [ ]:
start = {'2L':int(1e6),
        '2R':int(6e6),
        '3L':int(1e6),
        '3R':int(10e6)}
end = {'2L':int(17e6),
        '2R':int(19e6),
        '3L':int(17e6),
        '3R':int(26e6)}

In [ ]:
window_size = 10000

plt.figure(figsize=(15,10))
for i, chrom in enumerate(chromosomes):
    n_w = chrom_lengths[chrom] // window_size
    pos = np.arange(n_w)*window_size / 1e6
    
    hascov = nobs[chrom] > 0
    hascov_w = np.sum(hascov[:n_w*window_size].reshape(n_w, window_size), axis=1)
    
    pi = h.pairwise_diversity(mac[chrom], nobs[chrom])
    pi_w = np.nansum(pi[:n_w*window_size].reshape(n_w, window_size), axis=1)
                             
    plt.subplot(2,2,i+1)
    plt.plot(pos, pi_w/hascov_w, '.', ms=1, alpha=1)#, linestyle='steps-pre')
    plt.vlines([start[chrom]/1e6,end[chrom]/1e6], -0.001, 0.021)
    
    plt.ylim([-0.001, 0.021])

    plt.title(chrom)
    if i%2 == 0:
        plt.ylabel('Average pairwise diversity')
    if i >= 2:
        plt.xlabel('Position (Mb)')
plt.show()

In [ ]:
window_size = 10000

plt.figure(figsize=(15,10))
for i, chrom in enumerate(chromosomes):
    n_w = chrom_lengths[chrom] // window_size
    pos = np.arange(n_w)*window_size / 1e6
    nobs_w = np.mean(nobs[chrom][:n_w*window_size].reshape(n_w, window_size), axis=1)
                                 
    plt.subplot(2,2,i+1)
    plt.plot(pos, nobs_w, '.', ms=1, alpha=1)#, linestyle='steps-pre')
    
    plt.vlines([start[chrom]/1e6,end[chrom]/1e6], 0, 100)

    plt.title(chrom)
    if i%2 == 0:
        plt.ylabel('Avg. number of called genotypes')
    if i >= 2:
        plt.xlabel('Position (Mb)')
plt.show()

# Coverage statistics in the "central regions"

In [ ]:
nobs_c = {c:nobs[c][start[c]:end[c]] for c in chromosomes}
mac_c = {c:mac[c][start[c]:end[c]] for c in chromosomes}

In [ ]:
plt.figure(figsize=(15,10))
for i, chrom in enumerate(chromosomes):
    plt.subplot(2,2,i+1)
    plt.hist(nobs_c[chrom], bins=np.arange(0,101,1), cumulative=True, histtype='step')
    plt.xlim([0,100])
    plt.title(chrom)
    if i%2 == 0:
        plt.ylabel('Number of sites')
    if i >= 2:
        plt.xlabel('Number of genotypes')
plt.show()

In [ ]:
cov_cutoff = 90
print('Fraction of sites with >= 90 genotypes')
print('Chrom.\tAll\tPolymorphic')
for c in chromosomes:
    f_suf = np.mean(nobs_c[c] >= cov_cutoff)
    f_seg_suf = np.sum((nobs_c[c] >= cov_cutoff) & (mac_c[c]>0)) / np.sum(mac_c[c]>0)
    print('{}\t{:.3f}\t{:.3f}'.format(c, f_suf, f_seg_suf))

In [ ]:
sufficient_cov = {c:(nobs_c[c] >= cov_cutoff) for c in chromosomes}

In [ ]:
lag_max = 5000
sc_corr = {c: np.correlate(sufficient_cov[c].astype(int), sufficient_cov[c][:-lag_max].astype(int)) / len(sufficient_cov[c] - lag_max)
           for c in chromosomes}

In [ ]:
plt.figure(figsize=(15,10))
for i, c in enumerate(chromosomes):
    plt.subplot(2,2,i+1)
    y = sc_corr[c] - np.mean(sufficient_cov[c])**2
    plt.plot(np.arange(1,lag_max+1), y[1:]/y[0])
    plt.ylim([0,1])
    plt.xscale('log')
    plt.title(c)
    if i%2 == 0:
        plt.ylabel('Sufficient cov. autocorr.')
    if i >= 2:
        plt.xlabel('Distance (bp)')
plt.show()

It looks like there are two scales: order 10 bp and 1 Kb. The are the same for all chromosomes.

In [ ]:
window_size = 10

plt.figure(figsize=(15,10))
for i, chrom in enumerate(chromosomes):
    n_w = (end[chrom] - start[chrom]) // window_size
    sc_w = np.sum(sufficient_cov[chrom][:n_w*window_size].reshape(n_w, window_size), axis=1)

    plt.subplot(2,2,i+1)
    plt.hist(sc_w, bins=np.arange(0,window_size+1,1))

    plt.title(chrom)
    #if i%2 == 0:
    #    plt.ylabel('Avg. number of called genotypes')
   # if i >= 2:
    #    plt.xlabel('Position (Mb)')
plt.show()

In [ ]:
window_size = 100

plt.figure(figsize=(15,10))
for i, chrom in enumerate(chromosomes):
    n_w = (end[chrom] - start[chrom]) // window_size
    sc_w = np.sum(sufficient_cov[chrom][:n_w*window_size].reshape(n_w, window_size), axis=1)

    plt.subplot(2,2,i+1)
    plt.hist(sc_w, bins=np.arange(0,window_size+1,1))

    plt.title(chrom)
    #if i%2 == 0:
    #    plt.ylabel('Avg. number of called genotypes')
   # if i >= 2:
    #    plt.xlabel('Position (Mb)')
plt.show()

The second mode around 95 suggests that there are ~5bp elements that are being masked by the calling pipeline. This is probably contributing to the first (short) autocorrelation scale.

In [ ]:
window_size = 1000

plt.figure(figsize=(15,10))
for i, chrom in enumerate(chromosomes):
    n_w = (end[chrom] - start[chrom]) // window_size
    sc_w = np.sum(sufficient_cov[chrom][:n_w*window_size].reshape(n_w, window_size), axis=1)

    plt.subplot(2,2,i+1)
    plt.hist(sc_w, bins=np.arange(0,window_size+1,10))

    plt.title(chrom)
    #if i%2 == 0:
    #    plt.ylabel('Avg. number of called genotypes')
   # if i >= 2:
    #    plt.xlabel('Position (Mb)')
plt.show()

We may want to mask 1 Kb windows with fewer that 800 sufficent sites, but I'm not going to for now.

# Diversity correlations

In [ ]:
nobs_suf = {}
mac_suf = {}
maf = {}
pi = {}
for c in chromosomes:
    n = np.zeros_like(nobs_c[c])
    n[sufficient_cov[c]] = nobs_c[c][sufficient_cov[c]]
    nobs_suf[c] = n
    
    m = np.zeros_like(mac_c[c])
    m[sufficient_cov[c]] = mac_c[c][sufficient_cov[c]]
    mac_suf[c] = m
    
    f = m/n
    f[np.isnan(f)] = 0
    maf[c] = f
    
    p = h.pairwise_diversity(m,n)
    p[np.isnan(p)] = 0
    pi[c] = p

In [ ]:
for c in chromosomes:
    print('{}\t{:.4f}\t{:.3f}'.format(c,
                                      np.sum(pi[c])/np.sum(sufficient_cov[c]),
                                      np.sum(mac_suf[c]>0)/np.sum(sufficient_cov[c])))

In [ ]:
sufcov4d = {}
nobs4d = {}
mac4d = {}
maf4d = {}
pi4d = {}
for c in chromosomes:
    # Adjust the positions
    pos4d = fourD_pos[c][(fourD_pos[c]>start[c]) & (fourD_pos[c]<end[c])] - start[c]
    is4d = np.zeros_like(sufficient_cov[c])
    is4d[pos4d] = True
    print(np.mean(is4d))
    
    # This holds boolean values: True for sufficient coverage and 4d
    s = np.zeros_like(sufficient_cov[c])
    s[is4d] = sufficient_cov[c][is4d]
    sufcov4d[c] = s.astype(int)
    
    n = np.zeros_like(nobs_c[c])
    n[s] = nobs_c[c][s]
    nobs4d[c] = n
    
    m = np.zeros_like(mac_c[c])
    m[s] = mac_c[c][s]
    mac4d[c] = m
    
    f = m/n
    f[np.isnan(f)] = 0
    maf4d[c] = f
    
    p = h.pairwise_diversity(m,n)
    p[np.isnan(p)] = 0
    pi4d[c] = p
#    n = np.zeros_like(nobs_suf[c])
#    n[is4d] = nobs_suf[c][is4d]
#    nobs4d[c] = n
#    sufcov4d[c] = n>0
    
#    m = np.zeros_like(mac_suf[c])
#    m[is4d] = mac_suf[c][is4d]
#    mac4d[c] = m
    
#    f = m/n
#    f[np.isnan(f)] = 0
#    maf4d[c] = f
    
#    p = h.pairwise_diversity(m,n)
#    p[np.isnan(p)] = 0
#    pi4d[c] = p

In [ ]:
for c in chromosomes:
    print('{}\t{:.4f}\t{:.3f}'.format(c,
                                      np.sum(pi4d[c])/np.sum(sufcov4d[c]),
                                      np.sum(mac4d[c]>0)/np.sum(sufcov4d[c])))

In [ ]:
lag_max = 10000
comparisons4d = {}
for c in chromosomes:
    comparisons4d[c] = np.correlate(sufcov4d[c], sufcov4d[c][:-lag_max])

In [ ]:
print(comparisons4d)

In [ ]:
pi_mean = {c: np.sum(pi4d[c])/np.sum(sufcov4d[c])
           for c in chromosomes}
print(pi_mean)

In [ ]:
pi_corr4d = {c: np.correlate(pi4d[c], pi4d[c][:-lag_max])
           for c in chromosomes}

In [ ]:
for c in chromosomes:
    plt.semilogx(np.arange(3,lag_max,3), (pi_corr4d[c][3::3]/comparisons4d[c][3::3] /pi_mean[c]**2) - 1, '.')
    plt.hlines(0, 1, lag_max, linestyle='dashed')
    plt.show()

In [ ]:
for c in chromosomes:
    plt.semilogx(np.arange(3,lag_max,3), (pi_corr4d[c][3::3]/comparisons4d[c][3::3] /pi_mean[c]**2) - 1, '.')
    plt.hlines(0, 1, lag_max, linestyle='dashed')
    plt.show()

In [ ]:
freq_cutoff = 0.05
hi_freq4d = {c: ((mac4d[c]>0) & (maf4d[c] > freq_cutoff)).astype(int) for c in chromosomes}
lo_freq4d = {c: ((mac4d[c]>0) & (maf4d[c] <= freq_cutoff)).astype(int) for c in chromosomes}

In [ ]:
f_hi4d = {c: np.sum(hi_freq4d[c]) / np.sum(sufcov4d[c]) for c in chromosomes}
f_lo4d = {c: np.sum(lo_freq4d[c]) / np.sum(sufcov4d[c]) for c in chromosomes}
print(f_lo4d, f_hi4d)

In [ ]:
hilo_corr4d = {c: np.correlate(hi_freq4d[c], lo_freq4d[c][:-lag_max]) for c in chromosomes}

In [ ]:
for c in chromosomes:
    plt.semilogx(np.arange(3,lag_max,3), hilo_corr4d[c][3::3]/comparisons4d[c][3::3]/(f_lo4d[c]*f_hi4d[c]) - 1, '.')
    plt.hlines(0, 1, lag_max, linestyle='dashed')
    plt.show()